## Interface

The type of data expected by a function must be specified in the function parameters<br>
Don't always know the type ahead of time<br>
Interfaces allow specifying behaviors of a type instead of the type itself<br>
This allows functions to operate on more than one type of data
 

 Interfaces are implicitly implemented<br>
When a type has all receiver functions required by the
interface, then it is considered implemented<br>
Functions operating on interfaces should never accept a pointer to an interface<br>
Caller determines whether pointer or value (copy) is used<br>
Prefer multiple interfaces with a few functions over one large interface
 

Interfaces allow functions to operate on more than one data type<br>
Interfaces are implicitly implemented<br>
Create receiver functions matching interface function signatures<br>
No need to use pointers to interfaces in function parameters Use a pointer at the call site<br>
If a pointer receiver function is implemented, then the type can only be used as a pointer in function calls

In [5]:
package main

import "fmt"

type Preparer interface {
	PrepareDish()
}

type Chicken string
type Salad string

func (c Chicken) PrepareDish() {
	fmt.Println("cook chicken")
}

func (s Salad) PrepareDish() {
	fmt.Println("chop salad")
	fmt.Println("add dressing")
}

func prepareDishes(dishes []Preparer) {
	fmt.Println("Prepare dishes:")
	for i := 0; i < len(dishes); i++ {
		dish := dishes[i]
		fmt.Printf("--Dish: %v--\n", dish)
		dish.PrepareDish()
	}
	fmt.Println()
}

func main() {
	dishes := []Preparer{Chicken("Chicken Wings"), Salad("Iceberg Salad")}
	prepareDishes(dishes)

}

main()

ERROR: reflect.Value.Convert: value of type string cannot be converted to type *struct { 𒀪 xreflect.InterfaceHeader; PrepareDish func() }

In [6]:
//--Summary:
//  Create a program that directs vehicles at a mechanic shop
//  to the correct vehicle lift, based on vehicle size.
//
//--Requirements:
//* The shop has lifts for multiple vehicle sizes/types:
//  - Motorcycles: small lifts
//  - Cars: standard lifts
//  - Trucks: large lifts
//* Write a single function to handle all of the vehicles
//  that the shop works on.
//* Vehicles have a model name in addition to the vehicle type:
//  - Example: "Truck" is the vehicle type, "Road Devourer" is a model name
//* Direct at least 1 of each vehicle type to the correct
//  lift, and print out the vehicle information.
//
//--Notes:
//* Use any names for vehicle models

package main

import "fmt"

//* The shop has lifts for multiple vehicle sizes/types:
//  - Motorcycles: small lifts
//  - Cars: standard lifts
//  - Trucks: large lifts
const (
	SmallLift = iota
	StandardLift
	LargeLift
)

type Lift int

type LiftPicker interface {
	PickLift() Lift
}

type Motorcycle string
type Car string
type Truck string

//* Vehicles have a model name in addition to the vehicle type:
func (m Motorcycle) String() string {
	return fmt.Sprintf("Motorcycle: %v", string(m))
}

//* Vehicles have a model name in addition to the vehicle type:
func (c Car) String() string {
	return fmt.Sprintf("Car: %v", string(c))
}

//* Vehicles have a model name in addition to the vehicle type:
func (t Truck) String() string {
	return fmt.Sprintf("Truck: %v", string(t))
}

func (m Motorcycle) PickLift() Lift {
	return SmallLift
}

func (c Car) PickLift() Lift {
	return StandardLift
}

func (t Truck) PickLift() Lift {
	return LargeLift
}

//* Write a single function to handle all of the vehicles
//  that the shop works on.
func sendToLift(p LiftPicker) {
	switch p.PickLift() {
	case SmallLift:
		fmt.Printf("send %v to small lift\n", p)
	case StandardLift:
		fmt.Printf("send %v to standard lift\n", p)
	case LargeLift:
		fmt.Printf("send %v to large lift\n", p)
	}
}

func main() {
	car := Car("Sporty")
	truck := Truck("MountainCrusher")
	motorcycle := Motorcycle("Croozer")
	//* Direct at least 1 of each vehicle type to the correct
	//  lift, and print out the vehicle information.
	sendToLift(car)
	sendToLift(truck)
	sendToLift(motorcycle)
}




## Error Handling

Go has no exceptions <br>
Errors are returned as the last return value from a function <br>
Encodes failure as part of the function signature <br>
Simple to determine if a function can fail
Return nil if no error occurred<br>
Errors implement the error interface from `std`<br>
One function to implement: Error() string

In [7]:
package main

import (
	"errors"
	"fmt"
)

type Stuff struct {
	values []int
}

func (s *Stuff) Get(index int) (int, error) {
	if index > len(s.values) {
		return 0, errors.New(fmt.Sprintf("no element at index %v", index))
	} else {
		return s.values[index], nil
	}
}

func main() {
	stuff := Stuff{}
	value, err := stuff.Get(1)
	if err != nil {
		fmt.Println(err)
	} else {
		fmt.Println("value is", value)
	}
}

main()

no element at index 1


## Readers and Writers

 Reader & Writer are interfaces that allow reading from and writing to I/O sources<br>
Network sockets, files, arbitrary arrays<br>
Multiple implementations in standard library<br>
Reader is a low-level implementation<br>
Usually want to work with bufio package instead of Reader directly

In [9]:
//--Summary:
//  Create an interactive command line application that supports arbitrary
//  commands. When the user enters a command, the program will respond
//  with a message. The program should keep track of how many commands
//  have been ran, and how many lines of text was entered by the user.
//
//--Requirements:
//* When the user enters either "hello" or "bye", the program
//  should respond with a message after pressing the enter key.
//* Whenever the user types a "Q" or "q", the program should exit.
//* Upon program exit, some usage statistics should be printed
//  ('Q' and 'q' do not count towards these statistics):
//  - The number of non-blank lines entered
//  - The number of commands entered
//
//--Notes
//* Use any Reader implementation from the stdlib to implement the program

package main

import (
	"bufio"
	"fmt"
	"os"
	"strings"
)

const (
	CmdHello   = "hello"
	CmdGoodbye = "bye"
)

func main() {
	//* Use any Reader implementation from the stdlib to implement the program
	scanner := bufio.NewScanner(os.Stdin)
	numLines := 0
	numCommands := 0
	for scanner.Scan() {
		//* Whenever the user types a "Q" or "q", the program should exit.
		if strings.ToUpper(scanner.Text()) == "Q" {
			break
		} else {
			text := strings.TrimSpace(scanner.Text())
			//* When the user enters either "hello" or "bye", the program
			//  should respond with a message after pressing the enter key.
			switch text {
			case CmdHello:
				numCommands += 1
				fmt.Println("command response: hi")
			case CmdGoodbye:
				numCommands += 1
				fmt.Println("command response: bye")
			}
			if text != "" {
				numLines += 1
			}
		}
	}
	//* Upon program exit, some usage statistics should be printed
	//  ('Q' and 'q' do not count towards these statistics):
	//  - The number of non-blank lines entered
	//  - The number of commands entered
	fmt.Printf("You entered %v lines\n", numLines)
	fmt.Printf("You entered %v commands\n", numCommands)
}

main()

You entered 0 lines
You entered 0 commands


In [11]:
package main

import (
	"bufio"
	"fmt"
	"io"
	"os"
	"strconv"
	"strings"
)

func main() {
	scanner := bufio.NewReader(os.Stdin)

	sum := 0

	for {
		input, inputErr := scanner.ReadString(' ')

		num, convErr := strconv.Atoi(strings.TrimSpace(input))
		if convErr != nil {
			fmt.Println(convErr)
		} else {
			sum += num
		}

		if inputErr == io.EOF {
			break
		}
		if inputErr != nil {
			fmt.Println("Error reading Stdin:", inputErr)
		}
	}
	fmt.Printf("Sum: %v\n", sum)
}



## Type Embedding

Type embedding is a way to easily:<br>
Provide existing functionality to a new type<br>
Require a type to implement multiple interfaces

## Embedded Interfaces<br>

Embedded interfaces allow you to "embed" an interface into another interface<br>
Implementing the interface requires all embedded functions to be implemented<br>
Reduces the need to write duplicate interface declarations<br>
Changes in embedded interfaces automatically propagate<br>
Easier to maintain codebase (compiler errors will indicate where updates should be made)

In [12]:
package main

import "fmt"

const (
	Small = iota
	Medium
	Large
)

const (
	Ground = iota
	Air
)

type BeltSize int
type Shipping int

func (b BeltSize) String() string {
	return []string{"Small", "Medium", "Large"}[b]
}

func (s Shipping) String() string {
	return []string{"Ground", "Air"}[s]
}

type Conveyor interface {
	Convey() BeltSize
}

type Shipper interface {
	Ship() Shipping
}

type WarehouseAutomator interface {
	Conveyor
	Shipper
}

type SpamMail struct {
	amount int
}

func (s SpamMail) String() string {
	return "Spam mail"
}

func (s *SpamMail) Ship() Shipping {
	return Air
}

func (s *SpamMail) Convey() BeltSize {
	return Small
}

func automate(item WarehouseAutomator) {
	fmt.Printf("Convey %v on %v conveyor\n", item, item.Convey())
	fmt.Printf("Ship %v via %v\n", item, item.Ship())
}

type ToxicWaste struct {
	weight int
}

func (t *ToxicWaste) Ship() Shipping {
	return Ground
}

func main() {
	mail := SpamMail{40000}
	automate(&mail)

	// Using embedded interfaces to ensure only items that
	// implement both Conveyor and Shipper can be automated
	// automate(&ToxicWaste{300}) // Won't work!
}


ERROR: go/parser internal error: identifier already declared or resolved

In [14]:
//--Summary:
//  Create a system monitoring dashboard using the existing dashboard
//  component structures. Each array element in the components represent
//  a 1-second sampling.
//
//--Requirements:
//* Create functions to calculate averages for each dashboard component
//* Using struct embedding, create a Dashboard structure that contains
//  each dashboard component
//* Print out a 5-second average from each component using promoted
//  methods on the Dashboard

package main

import (
	"fmt"
)

type Bytes int
type Celcius float32

type BandwidthUsage struct {
	amount []Bytes
}

//* Create functions to calculate averages for each dashboard component
func (b *BandwidthUsage) AverageBandwidth() int {
	sum := 0
	for i := 0; i < len(b.amount); i++ {
		sum += int(b.amount[i])
	}
	return sum / len(b.amount)
}

type CpuTemp struct {
	temp []Celcius
}

func (c *CpuTemp) AverageCpuTemp() int {
	sum := 0
	for i := 0; i < len(c.temp); i++ {
		sum += int(c.temp[i])
	}
	return sum / len(c.temp)
}

type MemoryUsage struct {
	amount []Bytes
}

func (m *MemoryUsage) AverageMemoryUsage() int {
	sum := 0
	for i := 0; i < len(m.amount); i++ {
		sum += int(m.amount[i])
	}
	return sum / len(m.amount)
}

//* Using struct embedding, create a Dashboard structure that contains
//  each dashboard component
type Dashboard struct {
	BandwidthUsage
	CpuTemp
	MemoryUsage
}

func main() {
	bandwidth := BandwidthUsage{[]Bytes{50000, 100000, 130000, 80000, 90000}}
	temp := CpuTemp{[]Celcius{50, 51, 53, 51, 52}}
	memory := MemoryUsage{[]Bytes{800000, 800000, 810000, 820000, 800000}}

	dash := Dashboard{
		BandwidthUsage: bandwidth,
		CpuTemp:        temp,
		MemoryUsage:    memory,
	}

	//* Print out a 5-second average from each component using promoted
	//  methods on the Dashboard
	fmt.Printf("Average bandwidth usage: %v\n", dash.AverageBandwidth())
	fmt.Printf("Average temp: %v\n", dash.AverageCpuTemp())
	fmt.Printf("Average memory usage: %v\n", dash.AverageMemoryUsage())
}

main()

Average bandwidth usage: 90000
Average temp: 51
Average memory usage: 806000


Embedding interfaces allows multiple interfaces to be used as one<br>
Changes in embedded interfaces automatically propagate throughout codebase<br>
Embedding structs promotes the embedded struct's fields and methods<br>
Easy access to inner struct fields<br>
Receiver functions sharing the same name as promoted method will override the promoted method

## Function Literals<br>

Function literals provide a way to define a function within a function<br>
Possible to assign function literals to variables<br>
They can be passed to a function as parameters<br>
More dynamic code<br>
Also known as closures or anonymous functions<br>
Closures allow data to be encapsulated within

## Anonymous Function

In [15]:

func helloworld() {
    fmt.Printf("Hello,")
    
    world := func(){
        fmt.Printf("World!\n")
    }
    world()
    world()
    world()
}

 Function literals can be passed to other functions as arguments<br>
They can capture surrounding variables<br>
Type aliases are helpful when passing function literals to other functions<br>
Function literals can be returned from a function directly, or assigned to a variable<br>
Closure and anonymous functions are other terms for function literal

In [16]:
package main

import "fmt"

func add(lhs, rhs int) int {
	return lhs + rhs
}

func compute(lhs, rhs int, op func(lhs, rhs int) int) int {
	fmt.Printf("Running a computation with %v & %v\n", lhs, rhs)
	return op(lhs, rhs)
}

type Op func(lhs, rhs int) int

func computeTypedef(lhs, rhs int, op Op) int {
	fmt.Printf("Running a computation with %v & %v\n", lhs, rhs)
	return op(lhs, rhs)
}

func main() {
	fmt.Println("2 + 2 =", compute(2, 2, add))

	fmt.Println("10 - 2 =", compute(10, 2, func(lhs, rhs int) int {
		return lhs - rhs
	}))

	mul := func(lhs, rhs int) int {
		fmt.Printf("multiplying %v * %v = ", lhs, rhs)
		return lhs * rhs
	}
	fmt.Println(compute(3, 3, mul))
}


In [17]:
//--Summary:
//  Create a program that can create a report of rune information from
//  lines of text.
//
//--Requirements:
//* Create a single function to iterate over each line of text that is
//  provided in main().
//  - The function must return nothing and must execute a closure
//* Using closures, determine the following information about the text and
//  print a report to the terminal:
//  - Number of letters
//  - Number of digits
//  - Number of spaces
//  - Number of punctuation marks
//
//--Notes:
//* The `unicode` stdlib package provides functionality for rune classification

package main

import "fmt"
import "unicode"

type LineCallback func(line string)

//* Create a single function to iterate over each line of text that is
//  provided in main().
func lineIterator(lines []string, callback LineCallback) {
	for i := 0; i < len(lines); i++ {
		//  - The function must return nothing and must execute a closure
		callback(lines[i])
	}
}

func main() {
	lines := []string{
		"There are",
		"68 letters,",
		"five digits,",
		"12 spaces,",
		"and 4 punctuation marks in these lines of text!",
	}

	letters := 0
	numbers := 0
	punctuation := 0
	spaces := 0

	//* Using closures, determine the following information about the text and
	//  print a report to the terminal:
	//  - Number of letters
	//  - Number of digits
	//  - Number of spaces
	//  - Number of punctuation marks
	lineFunc := func(line string) {
		for _, r := range line {
			if unicode.IsLetter(r) {
				letters += 1
			}
			if unicode.IsDigit(r) {
				numbers += 1
			}
			if unicode.IsPunct(r) {
				punctuation += 1
			}
			if unicode.IsSpace(r) {
				spaces += 1
			}
		}
	}

	lineIterator(lines, lineFunc)

	// report
	fmt.Println(letters, "letters")
	fmt.Println(numbers, "digits")
	fmt.Println(spaces, "spaces")
	fmt.Println(punctuation, "punctuation marks")
}


In [18]:
main()

68 letters
5 digits
12 spaces
4 punctuation marks


## Defer
Useful to run operations after functions complete<br>
The defer keyword can be used to execute code after a function runs<br>
Clean up resources, reset data, etc

In [19]:
// How it works?

func one(){
    
    fmt.Println("1")
}

func two(){
    fmt.Println("2")
}


func three(){
    fmt.Println("3")
}

func sample(){
    
    fmt.Println("Begin")
    
    defer one()
    defer two()
    defer three()
    
    fmt.Println("End")
}

## Concurrency

 Code only executes line-by-line, one line at a time<br>
Concurrency allows multiple lines to be executed <br>
Two types of concurrent code:<br>
Threaded: code runs in parallel based on number of CPU cores<br>
Asynchronous: code can pause and resume execution<br>
While paused, other code can resume<br>
Go will automatically choose the appropriate concurrency method

 Single-threaded code runs deterministically <br>
Each run will produce the same result<br>
Concurrent code runs non-deterministically<br>
Code no longer executes line-by-line in a predefined order<br>
Cannot rely on results being the same each program run<br>
Extra care should be taken to ensure results are in order / sorted properly<br>
Accomplished using synchronization or by checking the final results in a single thread

 Concurrent code allows full utilization of available compute resources<br>
Go automatically abstracts threads and asynchronous operations<br>
Threaded code is used to make parallel computations on cores<br>
Asynchronous code may be paused/resumed and is used for waiting on resources (like networks)<br>
Concurrent code runs non-deterministically<br>
Synchronization or other techniques are required to ensure proper program behavior

## Goroutines<br>

Goroutines allow functions to run concurrently<br>
Can also run function literals / closures<br>
Go will automatically select parallel or asynchronous execution<br>
New goroutines can be created with the go keyword

In [20]:
package main

import "fmt"
import "time"
import "unicode"

func main() {
	data := []rune{'a', 'b', 'c', 'd'}
	var capitalized []rune

	capIt := func(r rune) {
		capitalized = append(capitalized, unicode.ToUpper(r))
		fmt.Printf("%c done!\n", r)
	}

	fmt.Printf("Before: %c\n", capitalized)
	for i := 0; i < len(data); i++ {
		go capIt(data[i])
	}
	time.Sleep(100 * time.Millisecond)
	fmt.Printf("After: %c\n", capitalized)
}

main()

Before: []
d done!
a done!
b done!
c done!
After: [B A C]


In [21]:
//--Summary:
//  Create a program to read a list of numbers from multiple files,
//  sum the total of each file, then sum all the totals.
//
//--Requirements:
//* Sum the numbers in each file noted in the main() function
//* Add each sum together to get a grand total for all files
//  - Print the grand total to the terminal
//* Launch a goroutine for each file
//* Report any errors to the terminal
//
//--Notes:
//* This program will need to be ran from the `lectures/exercise/goroutines`
//  directory:
//    cd lectures/exercise/goroutines
//    go run goroutines
//* The grand total for the files is 4103109
//* The data files intentionally contain invalid entries
//* stdlib packages that will come in handy:
//  - strconv: parse the numbers into integers
//  - bufio: read each line in a file
//  - os: open files
//  - io: io.EOF will indicate the end of a file
//  - time: pause the program to wait for the goroutines to finish

package main

import (
	"bufio"
	"fmt"
	"io"
	"os"
	"strconv"
	"time"
)

//* Sum the numbers in each file noted in the main() function
func sumFile(rd bufio.Reader) int {
	sum := 0
	for {
		line, err := rd.ReadString('\n')
		if err == io.EOF {
			return sum
		}
		if err != nil {
			//* Report any errors to the terminal
			fmt.Println("Error:", err)
		}
		// ReadString returns the delimiter as well (newline).
		// We need to get the string without the newline,
		// so we just slice the string up until the last byte.
		num, err := strconv.Atoi(line[:len(line)-1])
		if err != nil {
			//* Report any errors to the terminal
			fmt.Println("Error:", err)
		}
		sum += num
	}
}

func main() {
	files := []string{"num1.txt", "num2.txt", "num3.txt", "num4.txt", "num5.txt"}
	sum := 0

	for i := 0; i < len(files); i++ {
		file, err := os.Open(files[i])
		if err != nil {
			//* Report any errors to the terminal
			fmt.Println("Error:", err)
			return
		}

		rd := bufio.NewReader(file)

		calculate := func() {
			fileSum := sumFile(*rd)
			//* Add each sum together to get a grand total for all files
			sum += fileSum
		}
		//* Launch a goroutine for each file
		go calculate()
	}

	// Need to wait for goroutines to finish. Increase if needed.
	time.Sleep(100 * time.Millisecond)

	//  - Print the grand total to the terminal
	fmt.Println(sum)
}


## Channels

Channels offer one-way communication<br>
Conceptually the same as a two-ended pipe:<br>
Write data in one end and read data out the other<br>
This is also called sending and receiving<br>
Utilizing channels enables goroutines to communicate:<br>
Can send/receive messages or computational results<br>
Channel ends can be duplicated across goroutines

Channels can be buffered or unbuffered<br>
Unbuffered channels will block when sending until a reader is
available<br>
Buffered channels have a specified capacity<br>
Can send messages up to the capacity, even without a reader<br>
Messages on a channel are FIFO ordering

Channels are one-way communication pipes<br>
They have a send/write end and a receive/read end<br>
The ends of a channel can be duplicated across goroutines<br>
Bidirectional communication can be accomplished by using more channels<br>
select can be used to send or receive on multiple different channels<br>
Buffered channels are non-blocking, unbuffered channels will block

In [22]:
package main

import "fmt"
import "time"

type ControlMsg int

type Job struct {
	data   int
	result int
}

const (
	DoExit = iota
	ExitOk
)

func doubler(jobs, results chan Job, control chan ControlMsg) {
	for {
		select {
		case msg := <-control:
			switch msg {
			case DoExit:
				fmt.Println("Exit goroutine")
				control <- ExitOk
				return
			default:
				panic("Unhandled control message")
			}
		case job := <-jobs:
			results <- Job{data: job.data, result: job.data * 2}
		default:
			time.Sleep(50 * time.Millisecond)
		}
	}
}

func main() {
	jobs := make(chan Job, 50)
	results := make(chan Job, 50)
	control := make(chan ControlMsg)

	go doubler(jobs, results, control)

	for i := 0; i < 30; i++ {
		jobs <- Job{i, 0}
	}

	for {
		select {
		case result := <-results:
			fmt.Println(result)
		case <-time.After(500 * time.Millisecond):
			fmt.Println("timed out")
			control <- DoExit
			// Wait for response from goroutine before exit.
			<-control
			fmt.Println("program exit")
			return
		}
	}
}


In [23]:
//--Summary:
//  Create a program that utilizes goroutines to run the provided calculation
//  function on a number of jobs. The results from the goroutines must be
//  communicated back to the main thread using a channel, and then added
//  together.
//
//--Requirements:
//* Run `longCalculation` for each job generated by the `makeJobs` function
//* Each job must be run in a separate goroutine
//* The result from `longCalculation` must be provided to the main function
//  using a channel
//* Sum the results from each job to generate a final result, and print it
//  to the terminal

package main

import "fmt"
import "time"
import "math/rand"

type Job int

func longCalculation(i Job) int {
	duration := time.Duration(rand.Intn(1000)) * time.Millisecond
	time.Sleep(duration)
	fmt.Printf("Job %d complete in %v\n", i, duration)
	return int(i) * 30
}

func makeJobs() []Job {
	jobs := make([]Job, 0, 100)
	for i := 0; i < 100; i++ {
		jobs = append(jobs, Job(rand.Intn(10000)))
	}
	return jobs
}

func runJob(resultChan chan int, i Job) {
	//* The result from `longCalculation` must be provided to the main function
	//  using a channel
	resultChan <- longCalculation(i)
}

func main() {
	rand.Seed(time.Now().UnixNano())
	jobs := makeJobs()
	resultChan := make(chan int, 10)

	//* Run `longCalculation` for each job generated by the `makeJobs` function
	for i := 0; i < len(jobs); i++ {
		//* Each job must be run in a separate goroutine
		go runJob(resultChan, jobs[i])
	}

	resultCount := 0
	sum := 0
	for {
		result := <-resultChan
		//* Sum the results from each job to generate a final result, and print it
		//  to the terminal
		sum += result
		resultCount += 1
		if resultCount == len(jobs) {
			break
		}
	}

	fmt.Println("sum is", sum)
}


In [24]:
main()

Job 6042 complete in 15ms
Job 5541 complete in 24ms
Job 3802 complete in 41ms
Job 4698 complete in 57ms
Job 9808 complete in 57ms
Job 3526 complete in 79ms
Job 9008 complete in 81ms
Job 6439 complete in 82ms
Job 9488 complete in 90ms
Job 7565 complete in 112ms
Job 1025 complete in 119ms
Job 9357 complete in 126ms
Job 454 complete in 130ms
Job 6757 complete in 131ms
Job 9155 complete in 158ms
Job 7826 complete in 162ms
Job 5354 complete in 168ms
Job 7948 complete in 175ms
Job 8697 complete in 189ms
Job 9620 complete in 196ms
Job 7655 complete in 202ms
Job 6732 complete in 217ms
Job 6168 complete in 218ms
Job 3000 complete in 235ms
Job 7020 complete in 236ms
Job 7527 complete in 241ms
Job 8812 complete in 253ms
Job 1617 complete in 257ms
Job 4681 complete in 300ms
Job 8966 complete in 326ms
Job 4539 complete in 360ms
Job 5438 complete in 368ms
Job 8508 complete in 368ms
Job 8779 complete in 373ms
Job 1303 complete in 379ms
Job 8912 complete in 382ms
Job 7989 complete in 389ms
Job 9263 co

##  Synchronization

Managing data across multiple goroutines can become problematic and hard to debug<br>
Multiple goroutines can change the same data leading to unpredictable results<br>
Using channels to communicate is not always ideal<br>
Synchronization solves this issue, and enables:<br>
Waiting for goroutines to finish<br>
Prevents multiple goroutines from modifying data simultaneously

### Mutex<br>

 A mutex is short for mutual exclusion<br>
 Provides a way to lock and unlock data<br>
Locked data cannot be accessed by any other goroutine until it is unlocked<br>
While locked, all other goroutines are blocked until the mutex is unlocked<br>
Execution waits until lock is available, or if select is used<br>
Helps reduce bugs when working with multiple goroutines

## Wait Groups<br>
 Wait groups enable an application to wait for goroutines to finish<br>
Operates by incrementing a counter whenever a goroutine is added, and decrementing when it finishes<br>
Waiting on the group will block execution until the counter is 0

 Data can be safely accessed across goroutines using a mutex<br>
 Locking a mutex prevents other goroutines from locking it Always remember to unlock a mutex<br>
It is possible to wait for goroutines to finish with a wait group<br>
Add 1 per goroutine to the wait group, then use .Done() in
each goroutine to decrement the group counter<br>
Using defer makes it simple to unlock mutexes and when working with wait groups

In [25]:
//--Summary:
//  Create a program that can read text from standard input and count the
//  number of letters present in the input.
//
//--Requirements:
//* Count the total number of letters in any chosen input
//* The input must be supplied from standard input
//* Input analysis must occur per-word, and each word must be analyzed
//  within a goroutine
//* When the program finishes, display the total number of letters counted
//
//--Notes:
//* Use CTRL+D (Mac/Linux) or CTRL+Z (Windows) to signal EOF, if manually
//  entering data
//* Use `cat FILE | go run ./exercise/sync` to analyze a file
//* Use any synchronization techniques to implement the program:
//  - Channels / mutexes / wait groups

package main

import (
	"bufio"
	"fmt"
	"os"
	"strings"
	"sync"
	"unicode"
)

type Count struct {
	count int
	sync.Mutex
}

func getWords(line string) []string {
	return strings.Split(line, " ")
}

//* Count the total number of letters in any chosen input
func countLetters(word string) int {
	letters := 0
	for _, ch := range word {
		if unicode.IsLetter(ch) {
			letters += 1
		}
	}
	return letters
}

func main() {
	//* The input must be supplied from standard input
	scanner := bufio.NewScanner(os.Stdin)

	totalLetters := Count{}

	var wg sync.WaitGroup

	for {
		if scanner.Scan() {
			line := scanner.Text()
			words := getWords(line)
			for _, word := range words {
				wordCopy := word
				wg.Add(1)
				//* Input analysis must occur per-word, and each word must be analyzed
				//  within a goroutine
				go func() {
					totalLetters.Lock()
					defer totalLetters.Unlock()
					defer wg.Done()
					sum := countLetters(wordCopy)
					totalLetters.count += sum
				}()
			}
		} else {
			if scanner.Err() == nil {
				break
			}
		}
	}
	wg.Wait()

	//* When the program finishes, display the total number of letters counted
	totalLetters.Lock()
	sum := totalLetters.count
	totalLetters.Unlock()

	fmt.Println("total letters =", sum)
}


In [26]:
main()

total letters = 0
